Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [34]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
    
  
  # Training computation.
  
  intermediate = tf.matmul(tf_train_dataset, weights1) + biases1
  logits = tf.matmul(tf.nn.relu(intermediate), weights2) + biases2

  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 
        0.001 * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  intermediate_val = tf.matmul(tf_valid_dataset, weights1) + biases1
  logits_val = tf.matmul(tf.nn.relu(intermediate_val), weights2) + biases2
  valid_prediction = tf.nn.softmax(logits_val)
  #  tf.matmul(tf_valid_dataset, weights) + biases)

  intermediate_test = tf.matmul(tf_test_dataset, weights1) + biases1
  logits_test = tf.matmul(tf.nn.relu(intermediate_test), weights2) + biases2
  test_prediction = tf.nn.softmax(logits_test)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 329.949
Minibatch accuracy: 7.0%
Validation accuracy: 18.5%
Minibatch loss at step 500 : 47.2746
Minibatch accuracy: 82.0%
Validation accuracy: 59.6%
Minibatch loss at step 1000 : 33.8045
Minibatch accuracy: 85.2%
Validation accuracy: 62.5%
Minibatch loss at step 1500 : 32.7115
Minibatch accuracy: 83.6%
Validation accuracy: 62.3%
Minibatch loss at step 2000 : 26.5482
Minibatch accuracy: 85.2%
Validation accuracy: 62.4%
Minibatch loss at step 2500 : 27.1465
Minibatch accuracy: 82.8%
Validation accuracy: 61.9%
Minibatch loss at step 3000 : 25.7594
Minibatch accuracy: 89.1%
Validation accuracy: 63.0%
Test accuracy: 90.0%


**Test results**

`loss = loss_unreg
Minibatch accuracy: 82.8%
Validation accuracy: 64.0%
Test accuracy: 89.0%`

`loss = loss_unreg + 0.1 * tf.nn.l2_loss(weights2)  
Minibatch accuracy: 76.6%  
Validation accuracy: 58.2%  
Test accuracy: 84.4%  `

`loss = loss_unreg + 0.1 * (tf.nn.l2_loss(weights2) +  tf.nn.l2_loss(weights1))  
Minibatch accuracy: 80.5%  
Validation accuracy: 58.6%  
Test accuracy: 84.7%  `

`loss = loss_unreg + 0.001 * (tf.nn.l2_loss(weights2) +  tf.nn.l2_loss(weights1))  
Minibatch accuracy: 89.8%  
Validation accuracy: 71.6%  
Test accuracy: 93.3%  `

`loss = loss_unreg + 0.0001 * (tf.nn.l2_loss(weights2) +  tf.nn.l2_loss(weights1))  
Minibatch accuracy: 89.1%  
Validation accuracy: 63.0%  
Test accuracy: 90.0%`

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 4 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
    
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 653.406
Minibatch accuracy: 15.6%
Validation accuracy: 22.4%
Minibatch loss at step 4 : 1124.75
Minibatch accuracy: 45.3%
Validation accuracy: 38.5%
Test accuracy: 59.0%


`num_steps = 30 `
`Test accuracy: 82.0%
Test accuracy: 82.8%`


`num_steps = 21`
`Test accuracy: 83.8%
Test accuracy: 81.4%`

`num_steps = 5
Test accuracy: 62.8%
Test accuracy: 59.0`




---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [60]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
    
  
  # Training computation.
  
  first = tf.matmul(tf_train_dataset, weights1) + biases1
  intermediate = tf.nn.dropout(first,0.5)  
  logits = tf.matmul(tf.nn.relu(intermediate), weights2) + biases2

  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 
        0.001 * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  intermediate_val = tf.matmul(tf_valid_dataset, weights1) + biases1
  logits_val = tf.matmul(tf.nn.relu(intermediate_val), weights2) + biases2
  valid_prediction = tf.nn.softmax(logits_val)
  #  tf.matmul(tf_valid_dataset, weights) + biases)

  intermediate_test = tf.matmul(tf_test_dataset, weights1) + biases1
  logits_test = tf.matmul(tf.nn.relu(intermediate_test), weights2) + biases2
  test_prediction = tf.nn.softmax(logits_test)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [61]:
num_steps = 5

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 4 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
    
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 789.765
Minibatch accuracy: 14.8%
Validation accuracy: 23.4%
Minibatch loss at step 4 : 652.969
Minibatch accuracy: 50.8%
Validation accuracy: 42.4%
Test accuracy: 63.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [68]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))  
  
  global_step = tf.Variable(0) 

  # Training computation.
  
  first = tf.matmul(tf_train_dataset, weights1) + biases1
  intermediate = tf.nn.dropout(first,0.5)  
  logits = tf.matmul(tf.nn.relu(intermediate), weights2) + biases2

  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 
        0.001 * (tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))
    
    
   #tf.train.exponential_decay(learning_rate, global_step, 
    #decay_steps, decay_rate, staircase=False, name=None)
      
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.96, staircase=False)
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  intermediate_val = tf.matmul(tf_valid_dataset, weights1) + biases1
  logits_val = tf.matmul(tf.nn.relu(intermediate_val), weights2) + biases2
  valid_prediction = tf.nn.softmax(logits_val)
  #  tf.matmul(tf_valid_dataset, weights) + biases)

  intermediate_test = tf.matmul(tf_test_dataset, weights1) + biases1
  logits_test = tf.matmul(tf.nn.relu(intermediate_test), weights2) + biases2
  test_prediction = tf.nn.softmax(logits_test)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [69]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
    
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 774.129
Minibatch accuracy: 10.9%
Validation accuracy: 20.9%
Minibatch loss at step 500 : 210.153
Minibatch accuracy: 81.2%
Validation accuracy: 62.2%
Minibatch loss at step 1000 : 121.482
Minibatch accuracy: 79.7%
Validation accuracy: 62.3%
Minibatch loss at step 1500 : 72.3026
Minibatch accuracy: 79.7%
Validation accuracy: 62.5%
Minibatch loss at step 2000 : 44.9501
Minibatch accuracy: 79.7%
Validation accuracy: 65.6%
Minibatch loss at step 2500 : 28.7603
Minibatch accuracy: 79.7%
Validation accuracy: 66.8%
Minibatch loss at step 3000 : 18.3158
Minibatch accuracy: 86.7%
Validation accuracy: 69.8%
Test accuracy: 91.6%


`num_steps = 5
Test accuracy: 67.8%`

`num_steps = 3001
Test accuracy: 92.1% (straircase = True)
Test accuracy: 91.9% (staircase = False)`

In [44]:
whos

Variable            Type         Data/Info
------------------------------------------
accuracy            function     <function accuracy at 0x1082ffaa0>
batch_data          ndarray      128x784: 100352 elems, type `float32`, 401408 bytes (392 kb)
batch_labels        ndarray      128x10: 1280 elems, type `float32`, 5120 bytes
batch_size          int          128
biases1             Variable     <tensorflow.python.ops.va<...>le object at 0x1384fb290>
biases2             Variable     <tensorflow.python.ops.va<...>le object at 0x13dd9a910>
f                   file         <closed file 'notMNIST.pi<...>mode 'rb' at 0x1082ee8a0>
feed_dict           dict         n=2
first               Tensor       Tensor("add:0", shape=Ten<...>n(1024)]), dtype=float32)
graph               Graph        <tensorflow.python.framew<...>ph object at 0x1384fca50>
image_size          int          28
intermediate        Tensor       Tensor("dropout/mul_1:0",<...>n(1024)]), dtype=float32)
intermediate_test   Tensor  

In [58]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  #beta = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
    
  weights2 = tf.Variable(
    tf.truncated_normal([1024, 1024]))
  biases2 = tf.Variable(tf.zeros([1024]))  
    
  weights3 = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases3 = tf.Variable(tf.zeros([num_labels]))  
    
  
  # Training computation.
  
  first = tf.matmul(tf_train_dataset, weights1) + biases1
  intermediate = tf.nn.dropout(first,0.5)  
  second = tf.matmul(tf.nn.relu(intermediate), weights2) + biases2
  intermediate2 = tf.nn.dropout(second,0.5)  
  logits = tf.matmul(tf.nn.relu(intermediate2), weights3) + biases3

  loss = tf.reduce_mean(
    (tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + 
        0.00 * (tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights1)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  
  intermediate_val = tf.matmul(tf_valid_dataset, weights1) + biases1
  intermediate_val2 = tf.matmul(tf.nn.relu(intermediate_val), weights2) + biases2
  logits_val = tf.matmul(tf.nn.relu(intermediate_val2), weights3) + biases3
  valid_prediction = tf.nn.softmax(logits_val)
  #  tf.matmul(tf_valid_dataset, weights) + biases)

  intermediate_test = tf.matmul(tf_test_dataset, weights1) + biases1
  intermediate_test2 = tf.matmul(tf.nn.relu(intermediate_test), weights2) + biases2
  logits_test = tf.matmul(tf.nn.relu(intermediate_test2), weights3) + biases3
  test_prediction = tf.nn.softmax(logits_test)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [59]:
num_steps = 10

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 4 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
    
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 14087.2
Minibatch accuracy: 8.6%
Validation accuracy: 10.1%
Minibatch loss at step 4 : 5.5053e+32
Minibatch accuracy: 13.3%
Validation accuracy: 10.0%


InvalidArgumentError: ReluGrad input is not finite. : Tensor had Inf and NaN values
	 [[Node: gradients/Relu_1_grad/Relu_1/CheckNumerics = CheckNumerics[T=DT_FLOAT, message="ReluGrad input is not finite.", _device="/job:localhost/replica:0/task:0/cpu:0"](dropout_1/mul_1)]]
Caused by op u'gradients/Relu_1_grad/Relu_1/CheckNumerics', defined at:
  File "/Users/enricofonda/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/enricofonda/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/enricofonda/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/enricofonda/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 589, in launch_instance
    app.start()
  File "/Users/enricofonda/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 405, in start
    ioloop.IOLoop.